In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import joblib
import torch
import torch.nn.functional as F
import re
import unicodedata
import contractions
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


load_path = "/Users/pablogandia/Desktop/ASIGNATURAS/IA SAMSUNG/Proyecto/App/ModelsBias"
model = AutoModelForSequenceClassification.from_pretrained(load_path)
tokenizer = AutoTokenizer.from_pretrained(load_path)
le = joblib.load(f"{load_path}/label_encoder.pkl")

2025-07-30 18:51:13.593048: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def clean_text(text):

    combined = f"{text}"
    combined = contractions.fix(combined)
    combined = unicodedata.normalize('NFD', combined)
    combined = ''.join(ch for ch in combined if unicodedata.category(ch) != 'Mn')
    combined = re.sub(r'https?://t\.co/\S+|pic\.twitter\.com/\S+', ' link_twitter ', combined)
    combined = re.sub(r'https?://\S+|www\.\S+', ' link ', combined)
    combined = re.sub(r'©.*$', ' ', combined, flags=re.MULTILINE)
    combined = re.sub(r'#\w+', ' ', combined)
    combined = re.sub(r'@\w+', ' ', combined)
    combined = ''.join(ch for ch in combined if ch.encode('ascii', 'ignore').decode('ascii') == ch)
    combined = re.sub(r'All rights reserved.*$', ' ', combined, flags=re.IGNORECASE|re.MULTILINE)
    combined = re.sub(r'[^A-Za-z\s]', ' ', combined)
    combined = combined.lower()

    return re.sub(r'\s+', ' ', combined).strip()

In [3]:
def action(text):

    full_text = clean_text(text)

    inputs = tokenizer(full_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    probs = F.softmax(logits, dim=1).squeeze().tolist()

    classes = le.classes_
    pred_idx = torch.argmax(logits, dim=1).item()
    pred_label = classes[pred_idx]

    result = {
        "prediction": pred_label,
        "probabilities": {cls: round(probs[i], 4) for i, cls in enumerate(classes)}
    }

    return result


In [4]:
title = "Trump Slams China in Fiery Speech, Vows to Restore American Greatness"

text = """In a thunderous rally attended by tens of thousands of supporters, former President Donald Trump delivered a blistering speech denouncing the Chinese Communist Party and vowing to bring back American jobs, power, and pride. 'China has been robbing us blind for decades,' Trump declared to roaring applause. 'And the Biden administration is letting it happen. Not on my watch!'

Trump blamed China for manipulating global markets, stealing U.S. technology, flooding America with cheap goods, and, once again, for unleashing COVID-19 on the world. He also accused American elites of selling out the country to Beijing in exchange for profits and influence.

'We need to take our country back from the globalists, the socialists, and the China-first politicians,' he said. 'America First is not just a slogan — it’s a mission. We are done being weak, we are done being controlled.'

The former president called for 100% tariffs on Chinese imports, a total decoupling of U.S. supply chains from China, and massive investment in American manufacturing and energy independence. He also proposed banning Chinese companies from purchasing American farmland, universities, and media outlets.

'We will not be a colony of China,' Trump thundered. 'We will dominate economically, militarily, and spiritually — because we are Americans, and we bow to no one.'

The crowd responded with chants of 'USA! USA!' and 'Trump 2024', as flags waved and merchandise flew off tables. Conservative commentators praised the speech as a 'rallying cry for national sovereignty', while left-leaning outlets dismissed it as xenophobic and incendiary.

But among Trump’s base, the message was clear: the fight isn’t over — and China won’t win."""


In [6]:
results = action(text)

print(f"Prediction: {results['prediction']}")
print()
print("Probabilities:")
print()
for label, prob in results['probabilities'].items():
    print(f"{label}: {prob:.4f}")

Prediction: right

Probabilities:

center: 0.0130
leaning-left: 0.0779
leaning-right: 0.1566
left: 0.0871
right: 0.6654
